In [1]:
import boto3
from datetime import datetime

from database import db
from models import EventRegistration, EventUpdate, Register, Deregister
from aws_models import AWSConnection
from config import config_data


In [2]:
#######################################################
############# Initializa the S3 resource ##############
#######################################################

s3_bucket = AWSConnection(bucket=config_data['BUCKET_NAME_TO'],
                          access_key=config_data['AWS_ACCESS_KEY'],
                          access_secret=config_data['AWS_ACCESS_SECRET'],
                          resource='s3')

#######################################################
##### Get all the folders in the S3 Loka Data lake ####
#######################################################

folders = s3_bucket.list_folders_s3_bucket()

#######################################################
### Get all the files name in the S3 Loka Data lake ###
#######################################################

filesname = dict()
for folder in folders:
    files = s3_bucket.list_sorted_object_s3_bucket(prefix= folder)
    filesname['folder'] = folder
    filesname['filesnames'] = files


#######################################################
##### Get all the files in the S3 Loka Data lake ######
#######################################################
stg_events = list()

for filename in filesname['filesnames']:
    file_content_json = s3_bucket.get_file_s3_bucket(
        filename=f'{filesname["folder"]}/{filename}')
    for event in file_content_json:
        stg_row = dict()
        if event['event'] == 'register':
            stg_row['event'] = event['event']
            stg_row['on'] = event['on']
            stg_row['id'] = event['data']['id']
            stg_row['register_at'] = event['at']
            stg_events.append(stg_row)
        
        if event['event'] == 'deregister':
            stg_row['event'] = event['event']
            stg_row['on'] = event['on']
            stg_row['id'] = event['data']['id']
            stg_row['deregister_at'] = event['at']
            stg_events.append(stg_row)

        if event['event'] == 'update':
            stg_row['event'] = event['event']
            stg_row['on'] = event['on']
            stg_row['id'] = event['data']['id']
            stg_row['lat'] = event['data']['location']['lat']
            stg_row['lng'] = event['data']['location']['lng']
            stg_row['at'] = event['data']['location']['at']
            stg_row['organization_id'] = event['organization_id']
            stg_events.append(stg_row)

Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Status - 200
Successful S3 get_object response. Statu

In [3]:
register_list = []
deregister_list = []
event_registration = []
event_updates = []
locations = []

for element in stg_events:
    if element['event'] == 'register':
        instance_register = Register(element['id'], element['register_at'])
        register_list.append(instance_register)

    if element['event'] == 'deregister':
        instance_deregister = Deregister(element['id'], element['deregister_at'])
        deregister_list.append(instance_deregister)

    if element['event'] == 'update':
        instance_registration = EventRegistration(
            event = element['event'],
            on = element['on'],
            id = element['id'],
            register_at= None,
            deregister_at= None,
            organization_id= element['organization_id'])
        
        event_registration.append(instance_registration)

        instance_update = EventUpdate(
            element['id'],
            element['lat'],
            element['lng'],
            element['at']
        )

        event_updates.append(instance_update)

for eventRegistration in event_registration:
    for register in register_list:
        if eventRegistration.id == register.id:
            eventRegistration.register_at = register.register_at
    for deregister in deregister_list:
        if eventRegistration.id == deregister.id:
            eventRegistration.deregister_at = deregister.deregister_at


In [12]:
registration_seen_id = set()
unique_events_registration = []
for obj in event_registration:
    if obj.id not in registration_seen_id:
        unique_events_registration.append(obj)
        registration_seen_id.add(obj.id)   

In [ ]:
unique_events_registration

In [5]:
event_registration[35148].to_json()

{'event': 'update',
 'on': 'vehicle',
 'id': 'd57e9c3e-3479-47cd-9b20-9e317616a3f0',
 'register_at': '2019-06-01T18:17:04.089Z',
 'deregister_at': '2019-06-01T18:29:05.204Z',
 'organization_id': 'org-id'}

In [6]:
event_registration[35147].to_json()

{'event': 'update',
 'on': 'vehicle',
 'id': 'd57e9c3e-3479-47cd-9b20-9e317616a3f0',
 'register_at': '2019-06-01T18:17:04.089Z',
 'deregister_at': '2019-06-01T18:29:05.204Z',
 'organization_id': 'org-id'}

In [ ]:
# event_updates[2].to_json()
event_registration[15].to_json()

In [7]:
for dateEvent in dates_list:
    dateEvent.data = []
    for eventUpdate in event_updates:
        if datetime.strptime(eventUpdate.register_at, '%Y-%m-%dT%H:%M:%S.%f%z').strftime('%Y-%m-%d') == dateEvent.date:
            dateEvent.data.append(eventUpdate)

In [14]:
file = {}
all_dates = []
for date in dates_list:
    all_dates.append(date.to_json())
file['Test'] = all_dates

In [ ]:
# for event in unique_events_registration:
#     db.add(event)
#     db.flush()
#     db.commit()
# db.close()

for update in event_updates:
    db.add(update)
    db.flush()
    db.commit()
db.close()

In [20]:
db.rollback()

2023-03-06 22:14:29,523 INFO sqlalchemy.engine.Engine ROLLBACK
